In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
data = pd.read_csv('Data/train_rosbank.csv')
data['TRDATETIME'] = pd.to_datetime(data['TRDATETIME'], format='%d%b%y:%X')
data.head()

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,target_flag,target_sum
0,01/10/2017,0,5200,NaN,810,2017-10-21 00:00:00,5023.000,POS,0,0.000
1,01/10/2017,0,6011,NaN,810,2017-10-12 12:24:07,20000.000,DEPOSIT,0,0.000
2,01/12/2017,0,5921,NaN,810,2017-12-05 00:00:00,767.000,POS,0,0.000
3,01/10/2017,0,5411,NaN,810,2017-10-21 00:00:00,2031.000,POS,0,0.000
4,01/10/2017,0,6012,NaN,810,2017-10-24 13:14:24,36562.000,C2C_OUT,0,0.000


In [3]:
data['channel_type'] = data['channel_type'].fillna('0')

In [4]:
# Пересчитаем валютные транзакции в рубли по курсам цб
mapping = {810: 1, 978: 65.87, 504: 6.5, 704: 0.0025, 981: 23, 985: 15.48,
           840: 58.31, 949: 15.97, 51: 4.3, 826: 75.11, 214: 1.23, 764: 1.7,
           203: 2.5, 702: 42, 360: 0.0042, 756: 59.26, 933: 30.20, 975: 33.68,
           36: 13.5, 191: 8.5, 784: 15.9, 980: 2.2, 124: 44.97, 398: 0.18,
           376: 15.7, 944: 33.81, 352: 0.52, 417: 0.85, 156: 8.6, 752: 6.8,
           392: 0.52, 484: 3.3, 634: 15.8, 188: 0.11, 643: 1, 348: 0.21, 356: 0.90,
           458: 13.5, 986: 18, 498: 3.5, 578: 7.1, 208: 8.9, 344: 7.5, 32: 65.87,
           410: 0.05, 788: 24, 480: 1.6, 604: 17.5, 941: 0.55, 144: 0.37, 946: 14.4,
           710: 4.52, 690: 4, 44: 20, 170: 0.02, 901: 1.9, 608: 1.15, 554: 41.5, 462: 3.7}
data['amount'] = data['amount'] * data['currency'].map(mapping)

In [5]:
%%time

# создаем переменную - общее количество транзакций по каждому клиенту (cl_id)
data['total_number_transact_cl_id'] = data.groupby('cl_id')['TRDATETIME'].transform(
    lambda x: x.count())
# создаем переменную - уникальное количество MCC-кодов по каждому клиенту (cl_id)
data['uniq_number_MCC_cl_id'] = data.groupby('cl_id')['MCC'].transform(
    lambda x: x.nunique())
# создаем переменную - уникальное количество MCC-кодов в каждом 
# периоде (PERIOD) по каждому клиенту (cl_id)
data['uniq_number_MCC_cl_id_period'] = data.groupby(
    ['cl_id', 'PERIOD'])['MCC'].transform(lambda x: x.nunique())
# создаем переменную - самый часто встречающийся 
# тип транзакции по каждому клиенту
data['trx_category_cl_id_mode'] = data.groupby(
    'cl_id')['trx_category'].transform(lambda x: x.value_counts().index[0])
# создаем переменную - разницу в днях между самой ранней и самой 
# последней датой транзакции (TRDATETIME) по каждому клиенту (cl_id)
data['diff_days_cl_id'] = data.groupby('cl_id')['TRDATETIME'].transform(
    lambda x: x.max() - x.min()).dt.days
# создаем переменную - разницу в днях между самой ранней и самой последней датой 
# транзакции (TRDATETIME) в каждом периоде (PERIOD) по каждому клиенту (cl_id)
data['diff_days_cl_id_period'] = data.groupby(
    ['cl_id', 'PERIOD'])['TRDATETIME'].transform(lambda x: x.max() - x.min()).dt.days
# создаем переменную - количество дней, прошедших с первой транзакции, 
# по каждой транзакции для каждого клиента
data['days_from_first_transaction'] = data.groupby(
    'cl_id')['TRDATETIME'].transform(lambda x: x - x.min()).dt.days
# создаем переменную - максимальную абсолютную разность в днях, прошедших 
# с первой транзакции, для каждого клиента
data['mad_of_days_from_first_transaction'] = data.groupby(
     'cl_id')['days_from_first_transaction'].transform(lambda x: x.mad())
# создаем временную переменную - тип транзакции, пополняет она или, наоборот, сокращает счет
data['trx_type'] = data.apply(
    lambda x: 1 if x['trx_category'] in ['DEPOSIT', 'C2C_IN', 'BACK_TRX'] else -1, axis=1)
# создаем переменную - сумму каждой транзакции с учетом 
# знака (пополнения или снятия) по каждому клиенту
data['amount_signed'] = data['amount'] * data['trx_type']
# создаем переменную - сумму всех транзакций по каждому клиенту
data['amount_sum'] = data.groupby('cl_id')['amount'].transform(lambda x: x.sum())
# создаем переменную - сумму всех транзакций по каждому клиенту с учетом знака (пополнения или снятия)
data['amount_signed_sum'] = data.groupby('cl_id')['amount_signed'].transform(lambda x: x.sum())
# создаем переменную - сумму транзакций снятия по каждому клиенту
data['amount_snyatie'] = (data['amount_sum'] - data['amount_signed_sum']) / 2
# создаем переменную - сумму транзакций пополнения по каждому клиенту
data['amount_popolnenie'] = (data['amount_sum'] - data['amount_snyatie'])

CPU times: user 42.7 s, sys: 1.11 s, total: 43.8 s
Wall time: 30.2 s


In [6]:
data.head(10)

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,target_flag,target_sum,total_number_transact_cl_id,uniq_number_MCC_cl_id,uniq_number_MCC_cl_id_period,trx_category_cl_id_mode,diff_days_cl_id,diff_days_cl_id_period,days_from_first_transaction,mad_of_days_from_first_transaction,trx_type,amount_signed,amount_sum,amount_signed_sum,amount_snyatie,amount_popolnenie
0,01/10/2017,0,5200,0,810,2017-10-21 00:00:00,5023.000,POS,0,0.000,5,5,4,POS,53,12,8,14.720,-1,-5023.000,64383.000,-24383.000,44383.000,20000.000
1,01/10/2017,0,6011,0,810,2017-10-12 12:24:07,20000.000,DEPOSIT,0,0.000,5,5,4,POS,53,12,0,14.720,1,20000.000,64383.000,-24383.000,44383.000,20000.000
2,01/12/2017,0,5921,0,810,2017-12-05 00:00:00,767.000,POS,0,0.000,5,5,1,POS,53,0,53,14.720,-1,-767.000,64383.000,-24383.000,44383.000,20000.000
3,01/10/2017,0,5411,0,810,2017-10-21 00:00:00,2031.000,POS,0,0.000,5,5,4,POS,53,12,8,14.720,-1,-2031.000,64383.000,-24383.000,44383.000,20000.000
4,01/10/2017,0,6012,0,810,2017-10-24 13:14:24,36562.000,C2C_OUT,0,0.000,5,5,4,POS,53,12,12,14.720,-1,-36562.000,64383.000,-24383.000,44383.000,20000.000
5,01/10/2017,1,5814,0,810,2017-10-16 00:00:00,380.000,POS,0,0.000,104,23,6,POS,92,18,89,17.724,-1,-380.000,324982.152,-134982.152,229982.152,95000.000
6,01/10/2017,1,5814,0,810,2017-10-10 00:00:00,378.000,POS,0,0.000,104,23,6,POS,92,18,83,17.724,-1,-378.000,324982.152,-134982.152,229982.152,95000.000
7,01/10/2017,1,5814,0,810,2017-10-16 00:00:00,199.000,POS,0,0.000,104,23,6,POS,92,18,89,17.724,-1,-199.000,324982.152,-134982.152,229982.152,95000.000
8,01/10/2017,1,5814,0,810,2017-10-11 00:00:00,400.000,POS,0,0.000,104,23,6,POS,92,18,84,17.724,-1,-400.000,324982.152,-134982.152,229982.152,95000.000
9,01/07/2017,1,5411,0,810,2017-07-26 00:00:00,598.000,POS,0,0.000,104,23,3,POS,92,12,7,17.724,-1,-598.000,324982.152,-134982.152,229982.152,95000.000


In [7]:
%%time

aggregations = { 
    'total_number_transact_cl_id': 'mean',
    'uniq_number_MCC_cl_id': 'mean',
    'uniq_number_MCC_cl_id_period': 'mean',
    'trx_category_cl_id_mode': lambda x: x.value_counts().index[0],
    'diff_days_cl_id': 'mean',
    'diff_days_cl_id_period': 'mean',
    'days_from_first_transaction': 'mean',
    'mad_of_days_from_first_transaction': 'mean',
    'amount_signed': 'mean',
    'amount_sum': 'mean',
    'amount_signed_sum': 'mean',
    'amount_snyatie': 'mean',
    'amount_popolnenie': 'mean',
    'target_flag': lambda x: x.value_counts().index[0]
}

data_agg = data.groupby('cl_id').agg(aggregations).reset_index()

CPU times: user 4.4 s, sys: 241 ms, total: 4.64 s
Wall time: 3.58 s


In [8]:
data_agg.head(10)

,cl_id,total_number_transact_cl_id,uniq_number_MCC_cl_id,uniq_number_MCC_cl_id_period,trx_category_cl_id_mode,diff_days_cl_id,diff_days_cl_id_period,days_from_first_transaction,mad_of_days_from_first_transaction,amount_signed,amount_sum,amount_signed_sum,amount_snyatie,amount_popolnenie,target_flag
0,0,5,5,3.400,POS,53,9.600,16.200,14.720,-4876.600,64383.000,-24383.000,44383.000,20000.000,0
1,1,104,23,14.423,POS,92,24.596,59.298,17.724,-1297.905,324982.152,-134982.152,229982.152,95000.000,0
2,5,142,34,16.338,POS,92,23.697,47.951,23.597,-1899.197,557666.027,-269686.027,413676.027,143990.000,1
3,9,39,5,3.410,POS,89,26.128,43.667,25.744,-6292.695,849315.090,-245415.090,547365.090,301950.000,0
4,10,463,36,23.469,POS,89,28.758,44.423,19.295,-543.724,1124343.990,-251743.990,688043.990,436300.000,0
5,11,217,39,20.724,POS,91,26.332,39.203,20.694,-318.963,433215.032,-69215.032,251215.032,182000.000,0
6,14,136,30,14.574,POS,92,20.706,57.529,22.541,-1053.604,448690.170,-143290.170,295990.170,152700.000,1
7,20,77,17,12.156,POS,73,25.000,43.948,19.588,-94.910,437308.090,-7308.090,222308.090,215000.000,0
8,21,124,35,18.048,POS,88,27.387,44.363,21.257,-817.142,338881.650,-101325.650,220103.650,118778.000,0
9,22,59,12,6.034,POS,66,16.169,25.237,20.236,-77.407,249767.000,-4567.000,127167.000,122600.000,0


In [9]:
data_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 15 columns):
cl_id                                 5000 non-null int64
total_number_transact_cl_id           5000 non-null int64
uniq_number_MCC_cl_id                 5000 non-null int64
uniq_number_MCC_cl_id_period          5000 non-null float64
trx_category_cl_id_mode               5000 non-null object
diff_days_cl_id                       5000 non-null int64
diff_days_cl_id_period                5000 non-null float64
days_from_first_transaction           5000 non-null float64
mad_of_days_from_first_transaction    5000 non-null float64
amount_signed                         5000 non-null float64
amount_sum                            5000 non-null float64
amount_signed_sum                     5000 non-null float64
amount_snyatie                        5000 non-null float64
amount_popolnenie                     5000 non-null float64
target_flag                           5000 non-null int64
dtyp

In [10]:
y = data_agg.pop('target_flag')
X = pd.get_dummies(data_agg)

In [11]:
import lightgbm as lgb
kf = KFold(n_splits=5, shuffle=True, random_state=42)
lgboost = lgb.LGBMClassifier(n_estimators=500, max_depth=1, learning_rate=0.1,
                             random_state=42)
cross_val_score(lgboost, X, y, cv=kf, scoring='roc_auc').mean()

/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


0.8503885270608806